In [1]:
import json
import pandas as pd

json_data = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\participant_data.json"

with open(json_data, 'r') as f:
    data = json.load(f)

# user_name 초기화를 위한 빈 리스트 생성
records = []

# 각 user_name에 대해 meta 정보와 dot_info 정보 추출 및 데이터프레임 생성
for user_name, user_data in data.items():
    meta_info = user_data["meta"]
    dot_info = user_data["dot_info"]
    for n_key, n_value in dot_info.items():
        for m_key, m_value in n_value.items():
            # 각 dot_info 항목에 meta 정보 및 user_name 추가, n, m_key : 파일명 -> User_n_m
            combined_data = {**meta_info, **m_value, "user_name": user_name, "n_key": n_key, "m_key": m_key}
            records.append(combined_data)

df = pd.DataFrame(records)

# 왼쪽, 오른쪽 눈 df화
df_right = df[['user_name', 'file_name_right', 'right_landmarks', 'label']]
df_left = df[['user_name', 'file_name_left', 'left_landmarks', 'label']]


In [2]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array

import numpy as np

folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\img"

files = glob.glob(os.path.join(folder_path, '*.jpg'))

# right파일, left파일 각각 분리
right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]


def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array


# First & Sceond Inpput
right_images = [load_and_preprocess_image(img_path) for img_path in right_images]
left_images = [load_and_preprocess_image(img_path) for img_path in left_images]

# Labels
right_labels = np.array(df_right['label'].tolist())
left_labels = np.array(df_left['label'].tolist())

labels = right_labels


print(right_labels[0])
print(left_labels[0])

[0.22424955 0.28144491]
[0.22424955 0.28144491]


In [3]:
# Third Input
def expand_landmarks(landmarks):
    if len(landmarks) == 2:
        return pd.Series({
            'x1': landmarks[0][0],
            'y1': landmarks[0][1],
            'x2': landmarks[1][0],
            'y2': landmarks[1][1]
        })



# LandMark
right_landmark = df_right['right_landmarks']
left_landmark = df_left['left_landmarks']

right_landmarks_expanded = df_right['right_landmarks'].apply(expand_landmarks)
left_landmarks_expanded = df_left['left_landmarks'].apply(expand_landmarks)

expanded_landmarks_combined = pd.concat([right_landmarks_expanded.add_suffix('_right'), left_landmarks_expanded.add_suffix('_left')], axis=1)
landmark = np.array(expanded_landmarks_combined)
landmark

array([[83, 65, 46, ..., 64, 46, 65],
       [82, 65, 46, ..., 65, 46, 66],
       [83, 65, 46, ..., 65, 46, 66],
       ...,
       [84, 64, 44, ..., 63, 44, 63],
       [84, 64, 44, ..., 64, 44, 64],
       [84, 64, 44, ..., 64, 44, 64]], dtype=int64)

In [4]:
from sklearn.model_selection import train_test_split

right_images_train, right_images_test, left_images_train, left_images_test, labels_train, labels_test, landmark_train, landmark_test = train_test_split(
    right_images, left_images, right_labels, landmark, test_size=0.4, shuffle=True, random_state=42
)

In [5]:
import tensorflow as tf
from keras import layers, Model, Input

def mish(x):
    return x * (tf.math.tanh(tf.math.softplus(x)))

x1 = right_images  # 이미지
x2 = left_images
x3 = landmark

y1 = labels  # 레이블
y2 = labels

# input1 모델
input1 = Input(shape=(128, 128, 3))
x1 = layers.Conv2D(32, kernel_size=(7, 7), activation=mish, padding='same')(input1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(32, kernel_size=(5, 5), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(32, kernel_size=(5, 5), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(64, kernel_size=(3, 3), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(128, kernel_size=(3, 3), activation=mish, padding='same')(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = layers.BatchNormalization()(x1)

x1 = layers.Flatten()(x1)  # Flatten 레이어

# input2 모델
input2 = Input(shape=(128, 128, 3))
x2 = layers.Conv2D(32, kernel_size=(7, 7), activation=mish, padding='same')(input2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)

x2 = layers.Conv2D(32, kernel_size=(5, 5), activation=mish, padding='same')(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)

x2 = layers.Conv2D(32, kernel_size=(5, 5), activation=mish, padding='same')(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)

x2 = layers.Conv2D(64, kernel_size=(3, 3), activation=mish, padding='same')(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)

x2 = layers.Conv2D(128, kernel_size=(3, 3), activation=mish, padding='same')(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)

x2 = layers.Flatten()(x2)  # Flatten 레이어

# input3 모델
input3 = Input(shape=(8,))
x3 = layers.Dense(128, activation=mish)(input3)
x3 = layers.BatchNormalization()(x3)

x3 = layers.Dense(16, activation=mish)(x3)
x3 = layers.BatchNormalization()(x3)

x3 = layers.Dense(16, activation=mish)(x3)
x3 = layers.BatchNormalization()(x3)


# 직렬 Concatenate
combined = layers.Concatenate()([x1, x2, x3])

# Fully Connected 층
fc_output = layers.Dense(8, activation=mish)(combined)
fc_output = layers.BatchNormalization()(fc_output)
final_output = layers.Dense(2, activation=None)(fc_output)

model = Model(inputs=[input1, input2, input3], outputs=final_output)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 128, 128, 32)      │           4,736 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 128, 128, 32)      │           4,736 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 128, 128, 32)      │             128 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_5         │ (None, 128, 128, 32)      │             128 │ conv2d_5[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 64, 64, 32)        │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_5               │ (None, 64, 64, 32)        │               0 │ batch_normalization_5[0][… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 64, 64, 32)        │          25,632 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_6 (Conv2D)             │ (None, 64, 64, 32)        │          25,632 │ max_pooling2d_5[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 64, 64, 32)        │             128 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_6         │ (None, 64, 64, 32)        │             128 │ conv2d_6[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 32, 32, 32)        │               0 │ batch_normalization_1[0][… │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_6               │ (None, 32, 32, 32)        │               0 │ batch_normalization_6[0][… │
│ (MaxPooling2D)                │                           │               

 Total params: 336,090 (1.28 MB)

 Trainable params: 334,602 (1.28 MB)

 Non-trainable params: 1,488 (5.81 KB)

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
import tensorflow as tf
from sklearn.metrics import accuracy_score
import numpy as np

right_images_train = np.array(right_images_train)
left_images_train = np.array(left_images_train)
landmark_train = np.array(landmark_train)
labels_train = np.array(labels_train)

right_images_test = np.array(right_images_test)
left_images_test = np.array(left_images_test)
landmark_test = np.array(landmark_test)
labels_test = np.array(labels_test)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.016)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='mean_squared_logarithmic_error', metrics=[tf.keras.metrics.MeanSquaredError()])

# 모델 학습
history = model.fit(
    [right_images_train, left_images_train, landmark_train],  # 입력 데이터 리스트
    labels_train,                       # 레이블
    epochs=40,                                # 에포크 수
    batch_size=512,                          # 배치 사이즈
    validation_data=([right_images_test, left_images_test, landmark_test], labels_test),  # 검증 데이터와 레이블
    verbose=2                                
)

# 모델 평가
loss, mse_metric = model.evaluate([right_images_test, left_images_test, landmark_test], labels_test, verbose=0)
print(f'Test Loss (MSLE): {loss}')
print(f'Test MSE (metric): {mse_metric}')

# 예측
predictions = model.predict([right_images_test, left_images_test, landmark_test])

# 평가 지표 계산
mse = mean_squared_error(labels_test, predictions)
mae = mean_absolute_error(labels_test, predictions)
r2 = r2_score(labels_test, predictions)

# 결과 출력
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

# 오차 계산
errors = labels_test - predictions

# 예측 값, 실제 값 및 오차 출력 (예시로 첫 5개 데이터만 출력)
for i in range(5):  # 예시로 첫 5개 데이터만 출력
    print(f'Actual Value:\n{labels_test[i]}')
    print(f'Predicted Value:\n{predictions[i]}')
    print(f'Error:\n{errors[i]}\n')

Epoch 1/40


MemoryError: Unable to allocate 2.34 GiB for an array with shape (12798, 128, 128, 3) and data type float32